# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tazacorte,28.6290,-17.9293,27.54,69,0,8.75,ES,1691683268
1,1,shumerlya,55.5005,46.4129,23.43,60,16,1.20,RU,1691683268
2,2,tha khanon,9.0302,98.9529,28.34,84,29,1.52,TH,1691683269
3,3,edinburgh of the seven seas,-37.0676,-12.3116,11.35,66,30,11.19,SH,1691683269
4,4,puerto ayora,-0.7393,-90.3518,26.98,98,79,1.34,EC,1691683184


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot=city_data_df.hvplot.points("Lng","Lat", geo=True, tiles="OSM", frame_width=800, frame_height=500, size="Humidity",scale=0.5, color="City")

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df=city_data_df.loc[(city_data_df["Max Temp"]<28) & (city_data_df["Max Temp"]>20) & (city_data_df["Humidity"]<80)& (city_data_df["Wind Speed"]<5) & (city_data_df["Cloudiness"]==0)]

# Drop any rows with null values
ideal_city_data_df.dropna()

# Display sample data
ideal_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
44,44,plettenberg bay,-34.0527,23.3716,22.45,36,0,2.02,ZA,1691683288
75,75,port elizabeth,-33.9180,25.5701,25.05,77,0,2.57,ZA,1691683170
205,205,east london,-33.0153,27.9116,20.52,77,0,4.63,ZA,1691683235
249,249,hailey,43.5196,-114.3153,21.60,51,0,2.57,US,1691683363
252,252,rakops,-21.0167,24.3333,20.37,24,0,3.55,BW,1691683363
328,328,laguna,38.4210,-121.4238,24.97,72,0,3.09,US,1691683386
340,340,calama,-22.4667,-68.9333,20.27,8,0,4.63,CL,1691683243
343,343,sainte-suzanne,-20.9061,55.6089,21.12,73,0,3.60,RE,1691683402
373,373,newman,37.3138,-121.0208,24.57,66,0,2.68,US,1691683375
382,382,helena valley west central,46.6629,-112.0604,21.25,63,0,2.57,US,1691683419


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=ideal_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5,"Hotel Name","")

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
44,plettenberg bay,ZA,-34.0527,23.3716,36,
75,port elizabeth,ZA,-33.9180,25.5701,77,
205,east london,ZA,-33.0153,27.9116,77,
249,hailey,US,43.5196,-114.3153,51,
252,rakops,BW,-21.0167,24.3333,24,
328,laguna,US,38.4210,-121.4238,72,
340,calama,CL,-22.4667,-68.9333,8,
343,sainte-suzanne,RE,-20.9061,55.6089,73,
373,newman,US,37.3138,-121.0208,66,
382,helena valley west central,US,46.6629,-112.0604,63,


In [6]:
 base_url = "https://api.geoapify.com/v2/places"
params = {"apiKey":geoapify_key}
response=requests.get(base_url+"/apiKey:"+geoapify_key).json
response

<bound method Response.json of <Response [401]>>

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key, "limit": 20
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    
    longitude = row["Lng"]
    #print(longitude)
    latitude = row ["Lat"]
    #print(latitude)
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
plettenberg bay - nearest hotel: Milkwood Manor
port elizabeth - nearest hotel: Waterford Hotel
east london - nearest hotel: No hotel found
hailey - nearest hotel: Fairfield Inn
rakops - nearest hotel: Rakops River Lodge
laguna - nearest hotel: Holiday Inn Express & Suites
calama - nearest hotel: Apart Hotel Geotel Calama
sainte-suzanne - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
helena valley west central - nearest hotel: Home2 Suites
karratha - nearest hotel: Karratha International Hotel
muynoq - nearest hotel: Lala
colonia - nearest hotel: Wasserturm Hotel Cologne
talant - nearest hotel: La Bonbonnière


,City,Country,Lat,Lng,Humidity,Hotel Name
44,plettenberg bay,ZA,-34.0527,23.3716,36,Milkwood Manor
75,port elizabeth,ZA,-33.9180,25.5701,77,Waterford Hotel
205,east london,ZA,-33.0153,27.9116,77,No hotel found
249,hailey,US,43.5196,-114.3153,51,Fairfield Inn
252,rakops,BW,-21.0167,24.3333,24,Rakops River Lodge
328,laguna,US,38.4210,-121.4238,72,Holiday Inn Express & Suites
340,calama,CL,-22.4667,-68.9333,8,Apart Hotel Geotel Calama
343,sainte-suzanne,RE,-20.9061,55.6089,73,No hotel found
373,newman,US,37.3138,-121.0208,66,No hotel found
382,helena valley west central,US,46.6629,-112.0604,63,Home2 Suites


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [12]:
%%capture --no-display

# Configure the map plot
map_plot2=hotel_df.hvplot.points("Lng","Lat", hover_cols=["Country","Hotel Name"], geo=True, tiles="OSM", frame_width=800, frame_height=500, size="Humidity",scale=0.5, color="City")

# Display the map
map_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)